In [4]:
from keras_preprocessing.image import img_to_array, load_img


In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1/255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

generator = datagen.flow_from_directory(
    r"C:\Users\haris\Downloads\Flowers-Dataset\flowers",
    target_size=(150,150),
    batch_size=16,
    class_mode='sparse'
)

Found 4317 images belonging to 5 classes.


In [6]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import time
%load_ext tensorboard

In [8]:
dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]
SIZE = 150
INPUT_SHAPE = (SIZE, SIZE, 3) 

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
      
      print(NAME)
      #model initialzation
      model = Sequential()

      #model definition
      model.add(Conv2D(layer_size, (3,3), input_shape = INPUT_SHAPE))
      model.add(Activation('relu'))  
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation('relu')) 
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten())

      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu')) 


      #final output layer
      model.add(Dense(5))
      model.add(Activation('softmax'))

      #settings
      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
      #fit
      model.fit(enerator, batch_size=64, epochs=25, callbacks=[tensorboard])

3-conv-64-nodes-1-dense-1667298161
Epoch 1/25
270/270 [==============================] - 61s 225ms/step - loss: 1.2395 - accuracy: 0.4524
Epoch 2/25
270/270 [==============================] - 60s 223ms/step - loss: 0.9921 - accuracy: 0.6148
Epoch 3/25
270/270 [==============================] - 59s 217ms/step - loss: 0.9486 - accuracy: 0.6291
Epoch 4/25
270/270 [==============================] - 60s 221ms/step - loss: 0.8800 - accuracy: 0.6588
Epoch 5/25
270/270 [==============================] - 60s 220ms/step - loss: 0.8532 - accuracy: 0.6683
Epoch 6/25
270/270 [==============================] - 60s 221ms/step - loss: 0.8259 - accuracy: 0.6815
Epoch 7/25
270/270 [==============================] - 59s 220ms/step - loss: 0.8130 - accuracy: 0.6845
Epoch 8/25
270/270 [==============================] - 61s 226ms/step - loss: 0.8004 - accuracy: 0.6824
Epoch 9/25
270/270 [==============================] - 60s 223ms/step - loss: 0.7572 - accuracy: 0.7072
Epoch 10/25
270/270 [=================

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 activation_5 (Activation)   (None, 148, 148, 64)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 activation_6 (Activation)   (None, 72, 72, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                  

In [10]:
model.save('Flwers.h5')

In [11]:
import cv2

CATEGORIES = ['Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip']

def prepare(filepath):
  IMG_SIZE = 150
  img_array = cv2.imread(filepath)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [12]:
prediction = model.predict([prepare(r"C:\Users\haris\Downloads\9818247_e2eac18894.jpg")])
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 73ms/step
Dandelion
